<a href="https://colab.research.google.com/github/rameshhpathak/DisasterAssessment/blob/master/segmentation_major.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
Basic_blocks = drive.CreateFile({'id':'1LIJahCBOgKswPjV3dCdjno6iQunL1JQN'})
FusionNet = drive.CreateFile({'id':'1tEd6iRLRH-aQO7dzZkkYnZ4sYDVSaNNz'})
UNet = drive.CreateFile({'id':'1Xf0xSo-CswXiuiUR_tc9IbvzRx-UWU7M'})

In [0]:
Basic_blocks.GetContentFile('Basic_blocks.py')
FusionNet.GetContentFile('FusionNet.py')
UNet.GetContentFile('UNet.py')

In [0]:
from FusionNet import *
from UNet import *
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# hyperparameters

batch_size = 5
img_size = 256
lr = 0.001
epoch = 100

In [43]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!ls '/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/Roads'

mergetrain


In [0]:
# input pipeline

img_dir = "/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/Roads"

img_data = dset.ImageFolder(root=img_dir, transform=transforms.Compose([
    transforms.Resize(size=img_size),
    transforms.CenterCrop(size=(img_size, img_size * 2)),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=5)

In [46]:
torch.cuda.get_device_name(0)


'Tesla K80'

In [47]:
generator = FusionGenerator(3, 3, 64).cuda()


------Initiating FusionNet------



In [48]:
try:
  generator = torch.load('/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/segmentation/model/fusionnet.pkl')
  print('\n-------model restored--------\n')
except:
  print('\n-------model not restored----\n')
  pass


-------model not restored----



In [0]:
recon_loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(generator.parameters(), lr= lr)

In [0]:
loc = '/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/segmentation/model/'

In [0]:
file = open(loc + 'fusionnet_mse_loss', 'w')

In [52]:
len(img_batch)

798

In [0]:
import matplotlib.pyplot as plt

In [0]:
loss_in_epoch = []
for i in range(epoch):
  total_loss = 0
  for _, (image, label) in enumerate(img_batch):
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3)

        gen_optimizer.zero_grad()

        x = Variable(satel_image).cuda(0)
        y_ = Variable(map_image).cuda(0)
        y = generator.forward(x)

        loss = recon_loss_func(y, y_)
        file.write(str(loss) + "\n")
        loss.backward()
        gen_optimizer.step()
        
        total_loss+=loss

        if _ % 800 == 0:
            print(i)
            print(loss)
            v_utils.save_image(x.cpu().data, loc + "result/original_image_{}_{}.png".format(i, _))
            v_utils.save_image(y_.cpu().data, loc + "result/label_image_{}_{}.png".format(i, _))
            v_utils.save_image(y.cpu().data, loc + "result/gen_image_{}_{}.png".format(i, _))
            torch.save(generator, loc + 'fusionnet.pkl')
  loss_in_epoch.append(total_loss/len(img_batch))
  
  if i % 20 == 0:
    plt.plot(loss_in_epoch)
    plt.save(loc + 'loss/loss_in_{}'.format(i))
  

   
           



0
tensor(0.1131, device='cuda:0', grad_fn=<MseLossBackward>)
